In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:

pd_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data = np.array(pd_data)

pd_data.head() # we have 785 columns, first is the "right" answer for our example, the rest is the data

m, n = data.shape

np.random.shuffle(data)


## The math I'll implement

I'll be making a simple NN with only one hidden layer that has 16 neurons. Simple ReLU function with softmax X cross-entropy loss combo (nanoblade ?)

Across $m$ training examples in one gradient step we have,
$a^0 = 784 \times m, a^1 = 16 \times m, a^2 = 10 \times m $ 
$W^1 = 16 \times 784, W^2 = 10 \times 16, b^1 = 16 \times 1, b^2 = 10 \times 1 $


I honestly don't feel like rewriting the math in Latex, so here's my notebook page that has this stuff: 

In [3]:
data_test = data[0:1000].T # we take the first 1000 rows, each row has one column with answer, the rest is pixel data


y = data_test[0] #these are the "correct" answers, 1x1000 array
x = data_test[1:] #this is the our data 

x = x / 255 
# for the i'th training example-- x[:, i] we have the answer y[i] 


data_train = data[1000:].T

ytrain = data_train[0]
xtrain = data_train[1:]

xtrain = xtrain / 255



In [4]:
def init_params():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return w1,b1,w2,b2

def ReLU(x):
    return np.maximum(x, 0)

def dReLU(x):
    return x > 0

def softmax(z):
    z_max = np.max(z, axis=0, keepdims=True)        #1xm
    exp_z = np.exp(z - z_max)                       # 10x m
    denom = np.sum(exp_z, axis=0, keepdims=True)    #1xm
    return exp_z / denom 
    
def fprop(w1,b1,w2,b2,x):
    z1 = w1.dot(x) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2) 
    return z1, a1, z2, a2

def one_hot(y):
    x = np.zeros((10, y.size))
    x[y, np.arange(y.size)] = 1   

    return x

def compute_loss(a2, Y):
    m    = Y.shape[1]
    loss = -np.sum(Y * np.log(a2 + 1e-8)) / m
    return loss
    
    
    

def backprop(w1,b1,w2 ,b2, z1, a1, z2, a2, x, y):
    m = x.shape[1]
    dz2 = a2 - y
    dw2 = 1/m * dz2.dot(a1.T)
    db2 = 1/m * (np.sum(dz2,axis=1, keepdims=True))

    dz1 = w2.T.dot(dz2)*dReLU(z1)
    dw1 = 1/m * dz1.dot(x.T)
    db1 = 1/m * (np.sum(dz1,axis=1, keepdims=True))
    
    
    return dw2, db2, dw1, db1

def step(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1    
    w2 = w2 - alpha * dw2  
    b2 = b2 - alpha * db2    
    return w1, b1, w2, b2


def accuracy(a2, y):
    p = np.argmax(a2, 0)
    return np.sum(p == y) / y.size


In [5]:
def train(x, y):
    _, m = x.shape 
    w1, b1, w2, b2 = init_params()

    hot_y = one_hot(y)
    for i in range(500):
        alpha = 0.1
        z1, a1, z2, a2 = fprop(w1,b1,w2,b2, x)
        
        dw2, db2, dw1, db1 = backprop(w1,b1,w2,b2, z1, a1, z2, a2, x, hot_y)

        w1, b1, w2, b2 = step(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        # if i%25 == 0:
        #     # print(compute_loss(a2, hot_y))
            
        #     print(accuracy(a2, y))
    return w1, b1, w2, b2
        

In [6]:
w1, b1, w2, b2 = train(xtrain, ytrain)

In [7]:
def predict(w1, b1, w2, b2, x, y):
    _,_,_,a2 = fprop(w1,b1,w2,b2, x)
    return accuracy(a2, y)

predict(w1, b1, w2, b2, x, y)

0.842